In [ ]:
#from src.retriever import Retriever
import os,subprocess
import sys
sys.path.append('/root/data_disk/src')

from FlagEmbedding import BGEM3FlagModel,FlagReranker
from dotenv import load_dotenv
from src.LLM_utils import LLM
from src.retriever import Retriever
from utils import set_proxy
load_dotenv()
%load_ext autotime

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value


#unset http_proxy && unset https_proxy

In [ ]:
retriever = Retriever(embedder = 'BAAI/bge-m3',reranker = 'BAAI/bge-reranker-v2-m3',collection_name = 'sales_qa')

In [ ]:
embedder = BGEM3FlagModel('BAAI/bge-m3', cahche_dir = './autodl-tmp/.cache/huggingface/hub',use_fp16=True)

In [ ]:
retriever_path = '/root/autodl-tmp/.cache/huggingface/hub/'

In [ ]:
BGEM3FlagModel(embedder,use_fp16=True)

In [ ]:

model_id = 'ep-20250213200344-crq6r'
#model_id = 'LLM-Research/Meta-Llama-3.1-8B-Instruct-AWQ-INT4'
gen_llm = LLM(model_id,
              api_key = os.getenv('ARK_API_KEY'),base_url = os.getenv('ARK_BASE_URL')
             )
#model_id = 'LLM-Research/Meta-Llama-3.1-8B-Instruct-AWQ-INT4'
gen_llm('hi')

In [ ]:
import os
import subprocess
from contextlib import contextmanager

@contextmanager
def set_proxy():
    
    # Run the bash command to source the network configuration and capture the proxy settings
    result = subprocess.run(
        'bash -c "source /etc/network_turbo && env | grep proxy"', 
        shell=True, capture_output=True, text=True
    )

    # Set the proxy environment variables from the command output
    proxies = {}
    for line in result.stdout.splitlines():
        if '=' in line:
            var, value = line.split('=', 1)
            os.environ[var] = value
            
    # Yield control back to the block of code using the context manager
    yield

    # Restore the original proxy environment variables
    for key, value in os.environ.items():
        if 'proxy' in key:
            os.environ.pop(key, None)  # Remove the key if it wasn't set originally
with set_proxy():
    print(1)
proxies = {key: os.environ.get(key) for key in os.environ if 'proxy' in key.lower()}
print(proxies)

In [ ]:
?BGEM3FlagModel

In [ ]:
env_vars = """
HF_ENDPOINT=https://hf-mirror.com
HF_HOME=/root/autodl-tmp/.cache/huggingface
LMDEPLOY_USE_MODELSCOPE=True
MODELSCOPE_CACHE=/root/autodl-tmp/.cache/modelscope/
ARK_API_KEY= 'a2d08cdb-3d6f-42c0-bd4e-4b64ad505eb1'
ARK_BASE_URL = 'https://ark.cn-beijing.volces.com/api/v3'
retriever_path = '/root/autodl-tmp/.cache/huggingface/hub/'
reranker_path = '/root/autodl-tmp/.cache/huggingface/hub/'
vs_path = /root/data_disk/car/
"""

# Write to .env file
with open(".env", "w") as f:
    f.write(env_vars)

In [ ]:
import os,subprocess
import sys
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history: list):
        return "", history + [{"role": "user", "content": user_message}]
        
    def bot(history: list):
        bot_message = random.choice([""""Hey! 😊 I’m here to make your car-buying experience smooth and stress-free. Tell me a bit about what you’re looking for, and I’ll get you started!""",
                                    """Hello and welcome! 🎉 Ready to find your dream car? Let me guide you through our latest models, special offers, and financing options. What are you looking for?""",
                                    """"Hey there! 👋 Excited to help you find the perfect car today! Whether it’s something sleek, spacious, or sporty, I’ve got you covered. What kind of vehicle are you dreaming of?"""])
        
        history.append({"role": "assistant", "content": ""})
        for character in bot_message:
            history[-1]['content'] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch()

In [ ]:
import os, sys
sys.path.append('/root/data_disk/src')

from retriever import Retriever
from LLM_utils import LLM
from prompt import generate_prompt

In [ ]:



query, history = 'i need a bmw', []

# Initialize the Retriever with specific embedder and reranker
retriever = Retriever(
    embedder='BAAI/bge-m3',
    reranker='BAAI/bge-reranker-v2-m3',
    collection_name='sales_qa'
)

# Initialize the first LLM for query rewriting
model_id = 'Qwen/Qwen2.5-3B-Instruct'
rewriter = LLM(model_id)

# Initialize the second LLM for response generation
model_id = 'ep-20250213200344-crq6r'
gen_llm = LLM(
    model_id,
    api_key=os.getenv('ARK_API_KEY'),  # Fetch API key from environment variables
    base_url=os.getenv('ARK_BASE_URL'),
    stream = True # Fetch base URL from environment variables
)

# Query rewriting
rewrite_prompt = generate_prompt('rewrite', query, history, reranked_chunks=None)
rewritten_query = rewriter(rewrite_prompt)

# Retrieval
recall_docs, reranked_chunks = retriever.retrieve(rewritten_query)

# Response generation
generation_prompt = generate_prompt('generate', query, history, reranked_chunks)
response = gen_llm(generation_prompt)

if isinstance(response, str):  
    print(response)  # Directly print if it's a normal response
else:  
    for chunk in response:  # Iterate if it's a generator
        print(chunk, end="", flush=True)  # Print chunks continuously
        
# Update history
history.append({"User": rewritten_query, "Assistant": response})

print(history)


In [ ]:
how do I combine the code for RAG QA into the gradio UI integration code below